In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/PLDD

Mounted at /content/drive
/content/drive/MyDrive/PLDD


In [7]:
import os
import pickle
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image



In [8]:
image_folder = 'custom_dataset/SRGAN_input/PlantVillage'
image_list = []
label_list = []
max_images_per_folder = 225

for folder in os.listdir(image_folder):
    folder_count = 0
    for image_file in os.listdir(os.path.join(image_folder, folder)):
        try:
            if folder_count >= max_images_per_folder:
                break
            image = Image.open(os.path.join(image_folder, folder, image_file))
            image_array = np.asarray(image)
            image_list.append(image_array)
            label_list.append(folder)
            folder_count += 1
        except:
            print(f"Deleted file: {os.path.join(image_folder, folder, image_file)}")


In [9]:
image_size = len(image_list)
image_size

3302

In [10]:
lable_size = len(label_list)
lable_size

3302

In [11]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('SRCNN_label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

In [12]:
print(label_binarizer.classes_)

['Pepper__bell___Bacterial_spot' 'Pepper__bell___healthy'
 'Potato___Early_blight' 'Potato___Late_blight' 'Potato___healthy'
 'Tomato_Bacterial_spot' 'Tomato_Early_blight' 'Tomato_Late_blight'
 'Tomato_Leaf_Mold' 'Tomato_Septoria_leaf_spot'
 'Tomato_Spider_mites_Two_spotted_spider_mite' 'Tomato__Target_Spot'
 'Tomato__Tomato_YellowLeaf__Curl_Virus' 'Tomato__Tomato_mosaic_virus'
 'Tomato_healthy']


In [13]:
np_image_list = np.array(image_list, dtype=np.float16) / 255.0

In [14]:
print("[INFO] Spliting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42)

[INFO] Spliting data to train, test


In [16]:
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load saved model
model = load_model('SRCNN MODELS/SRCNN_model.h5')
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


21/21 [==============================] - 18s 836ms/step
Accuracy: 0.886535552193646
Precision: 0.8887587385881587
Recall: 0.886535552193646
F1 Score: 0.8844039422314419
